<a href="https://colab.research.google.com/github/ismarano/IAEntregable2/blob/master/entregable3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entregable 3

Importamos las librerias de fastai y os

In [0]:
from fastai.vision import *
import os
from google.colab import files

Vamos a buscar las imagenes, en este caso de coches, motos y camiones. Creamos archivos csv de cada uno.

Descargamos los ficheros

In [7]:
!wget https://raw.githubusercontent.com/IA1819/entregable3-ismarano/master/coche.csv?token=AKN7FSGGXML4AOVP4FX7EG244WF42 -O coche.csv
!wget https://raw.githubusercontent.com/IA1819/entregable3-ismarano/master/moto.csv?token=AKN7FSE7LRMLG7RE6ZEHFHC44WFAS -O moto.csv
!wget https://raw.githubusercontent.com/IA1819/entregable3-ismarano/master/camion.csv?token=AKN7FSGB6FUXUX547UEWAN244WEY2 -O camion.csv

--2019-05-15 17:50:21--  https://raw.githubusercontent.com/IA1819/entregable3-ismarano/master/coche.csv?token=AKN7FSGGXML4AOVP4FX7EG244WF42
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9680 (9.5K) [text/plain]
Saving to: ‘coche.csv’

coche.csv           100%[===================>]   9.45K  --.-KB/s    in 0s      

2019-05-15 17:50:21 (77.2 MB/s) - ‘coche.csv’ saved [9680/9680]

--2019-05-15 17:50:22--  https://raw.githubusercontent.com/IA1819/entregable3-ismarano/master/moto.csv?token=AKN7FSE7LRMLG7RE6ZEHFHC44WFAS
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting res

Vamos a descargar las imagenes y ordenarlas por carpetas

In [8]:
folder = 'coche'
file = 'coche.csv'
path = Path('data/vehiculos')
dest = path/folder
dest.mkdir(parents=True,exist_ok=True)
os.rename(file,path/file)
# Como mucho descargamos 200 imágenes
download_images(path/file,dest,max_pics=200)

Error https://www.todococheszafra.com/wp-content/uploads/2018/08/logo2018.png ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))
Error https://s5.eestatic.com/2018/11/06/reportajes/Coche_electrico-Venta_de_coches-Medio_Ambiente-Salon_del_Automovil_de_Paris-Reportajes_351227643_104663659_1706x960.jpg HTTPSConnectionPool(host='s5.eestatic.com', port=443): Read timed out. (read timeout=4)


In [5]:
folder = 'moto'
file = 'moto.csv'
path = Path('data/vehiculos')
dest = path/folder
dest.mkdir(parents=True,exist_ok=True)
os.rename(file,path/file)
# Como mucho descargamos 200 imágenes
download_images(path/file,dest,max_pics=200)

Error https://www.pontgrup.com/img/seguros-moto/seguros-de-moto-deportivas.png HTTPSConnectionPool(host='www.pontgrup.com', port=443): Max retries exceeded with url: /img/seguros-moto/seguros-de-moto-deportivas.png (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f3fede58240>: Failed to establish a new connection: [Errno -5] No address associated with hostname',))


In [6]:
folder = 'camion'
file = 'camion.csv'
path = Path('data/vehiculos')
dest = path/folder
dest.mkdir(parents=True,exist_ok=True)
os.rename(file,path/file)
# Como mucho descargamos 200 imágenes
download_images(path/file,dest,max_pics=200)

Error https://www.irizar.com/wp-content/uploads/2019/03/JAZ_763549r.jpg HTTPSConnectionPool(host='www.irizar.com', port=443): Max retries exceeded with url: /wp-content/uploads/2019/03/JAZ_763549r.jpg (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f3fef4d6438>: Failed to establish a new connection: [Errno 111] Connection refused',))
Error https://www.autonomosenruta.com/images/stories/imagenes_2017/imagenes_noviembre17/camion_scania450.jpg ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))


Vamos a eliminar las imagenes que tengan un formato que no pueda ser abierto en la libreria

In [10]:
for c in ['coche','moto','camion']:
  print(c)
  verify_images(path/c,delete=True,max_size=500)

coche


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')
/usr/local/lib/python3.6/dist-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


cannot identify image file <_io.BufferedReader name='data/vehiculos/coche/00000118.svg'>
cannot identify image file <_io.BufferedReader name='data/vehiculos/coche/00000138.JPG'>
cannot identify image file <_io.BufferedReader name='data/vehiculos/coche/00000073.svg'>


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:967: UserWarning: Couldn't allocate palette entry for transparency
  "for transparency")


int() argument must be a string, a bytes-like object or a number, not 'tuple'
cannot identify image file <_io.BufferedReader name='data/vehiculos/coche/00000067.svg'>


...

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


cannot identify image file <_io.BufferedReader name='data/vehiculos/moto/00000043.jpg'>
cannot identify image file <_io.BufferedReader name='data/vehiculos/moto/00000071.jpg'>
camion


cannot identify image file <_io.BufferedReader name='data/vehiculos/camion/00000088.jpg'>
cannot identify image file <_io.BufferedReader name='data/vehiculos/camion/00000027.jpg'>
cannot identify image file <_io.BufferedReader name='data/vehiculos/camion/00000040.jpg'>


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')
/usr/local/lib/python3.6/dist-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


cannot identify image file <_io.BufferedReader name='data/vehiculos/camion/00000099.jpg'>
